In [4]:
import pandas as pd
import numpy as np
import boto3

from shapely.geometry.polygon import Polygon
from shapely.geometry import box
from shapely.geometry import Polygon, LineString, Point
import shapely.wkt
import geopandas as gpd
import json

import folium
import plotly.express as px

# filter

## only keep smartphone

In [ ]:
"""
create table m_hk_filter_v1 as
select * from m_hk
where device_category = 'SMART PHONE' and year(timestamp) = 2019
"""

"""
insert into m_hk_filter_v1 
select * from m_hk
where device_category = 'SMART PHONE' and year(timestamp) = 2020
"""

"""
insert into m_hk_filter_v1 
select * from m_hk
where device_category = 'SMART PHONE' and year(timestamp) = 2021
"""

"""
insert into m_hk_filter_v1 
select * from m_hk
where device_category = 'SMART PHONE' and year(timestamp) = 2022
"""

## Filter out the data that only have one wifi connection and ip address over time 

In [ ]:
# find ifa with nct > 1 and nip > 1
"""
create table m_hk_filter_v2 as
with df1 as (
select ifa, count(distinct connection_type) as nct, count(distinct ip_address) as nip from m_hk_filter_v1
group by ifa
)

select * from df1 where nct > 1 and nip > 1
"""

In [ ]:
"""
create table m_hk_filter_v1_2019 as(
select * from m_hk_filter_v1
where year(timestamp) = 2019
);
"""

"""
create table m_hk_filter_v1_2020 as(
select * from m_hk_filter_v1
where year(timestamp) = 2020
);
"""

"""
create table m_hk_filter_v1_2021 as(
select * from m_hk_filter_v1
where year(timestamp) = 2021
);
"""

In [ ]:
# merge data
"""
create table m_hk_filter_v2_1 as
select b.* from m_hk_filter_v2 a join (select * from m_hk_filter_v1 where (year(timestamp)=2019) and (month(timestamp) = 1)) b 
            on a.ifa = b.ifa
"""

# ...
"""
insert into m_hk_filter_v2_1 
select b.* from m_hk_filter_v2 a join (select * from m_hk_filter_v1 where (year(timestamp)=2022) and (month(timestamp) = 1)) b 
            on a.ifa = b.ifa
"""

## Clustered location

In [ ]:
# find the ifa with nlocation>1
"""
create table m_hk_filter_v3 as
with df1 as (
select *, round(latitude,3) as round_lat, round(longitude,4) as round_long from m_hk_filter_v2_1
),

df2 as(
select ifa, round_lat, round_long, count(*) from df1 
group by ifa, round_lat, round_long
),

df3 as(
select ifa, count(*) nlocation from df2
group by ifa
)


select * from df3
where nlocation>1
"""

In [ ]:
# merge data
"""
create table m_hk_filter_v3_1 as
select b.* from m_hk_filter_v3 a join 
(select * from m_hk_filter_v2_1 
where (year(timestamp) = 2019) and (month(timestamp) = 1)
) b
on a.ifa = b.ifa
"""

"""
insert into m_hk_filter_v3_1 
select b.* from m_hk_filter_v3 a join 
(select * from m_hk_filter_v2_1 
where (year(timestamp) = 2022) and (month(timestamp) = 1)
) b
on a.ifa = b.ifa
"""

### get distribution data

In [ ]:
"""
with df1 as (
select *, round(latitude,3) as round_lat, round(longitude,3) as round_long from m_hk_filter_v2_1
),

df2 as(
select ifa, round_lat, round_long, count(*) from df1 
group by ifa, round_lat, round_long
),

df3 as(
select ifa, count(*) nlocation from df2
group by ifa
)


select nlocation, count(distinct ifa) freq from df3
group by nlocation
order by nlocation, freq

"""

In [68]:
df = pd.read_csv('../data/15_clustered_location_round3_hist.csv')
fig = px.scatter(df, x='nlocation', y='freq')
fig.update_layout(xaxis_range=[0,1000])
fig.show()

In [103]:
df = pd.read_csv('../data/15_clustered_location_round4_hist.csv')
fig = px.scatter(df, x='nlocation', y='freq')
fig.update_layout(xaxis_range=[0,1000])
fig.show()

## keep ifa with > n days 

### get distribution data

In [ ]:
"""
with df1 as (
select ifa, count(distinct date(timestamp)) nday from m_hk_filter_v3_1
group by ifa)

select nday, count(distinct ifa) from df1
group by nday 

"""

In [90]:
df = pd.read_csv('../data/15_nday_count_ifa_hist.csv')

In [98]:
sum(df.loc[df['nday']>7,'_col1'])

5441164

In [101]:
fig = px.scatter(df, x='nday', y='_col1')
fig.show()

## keep daily points greater than n 

In [ ]:
"""
with df as(
select distinct ifa, date(timestamp) day, ST_AsText(st_point(longitude,latitude)) point from m_hk_filter_v3_1
),

df1 as (
select ifa, day, count(distinct point) nlocation from df
group by ifa, day)

select nlocation, count(distinct ifa) from df1
group by nlocation 
"""

In [105]:
df = pd.read_csv('../data/15_nlocation_count_ifa_hist.csv')

In [109]:
fig = px.scatter(df, x='nlocation', y='_col1')
fig.show()

# run athena code

In [25]:
with open('../aws1.txt') as f:
    lines = f.readlines()

client = boto3.client('athena', region_name='us-east-1',
    aws_access_key_id = lines[1].strip(' \n'),
    aws_secret_access_key= lines[4])

In [30]:
def query(year,month):
    return  f"""
            inset into m_hk_filter_v2_1
            select b.* from m_hk_filter_v2 a join (select * from m_hk_filter_v1 where year(timestamp) = {year} and month(timestamp) = {month}) b 
            on a.ifa = b.ifa 
            """

In [31]:
date = pd.DataFrame([list(np.repeat([2019,2020,2021],12))+[2022],list(range(1,13))*3+[1]]).T
date.columns = ['year','month']

In [33]:
for i in range(date.shape[0]):
    year = date.loc[i,'year']
    month = date.loc[i,'month']
    response = client.start_query_execution(
        QueryString=query(year,month),
        QueryExecutionContext={
            'Database': 'default'
        },
        ResultConfiguration={
            'OutputLocation': f's3://alantuan/m_summary/m_hk_filter_v2/dt_{year}_{month}'
        })
    print(f'{year} {month} is processing')


2022 1 Done


NameError: name 'time' is not defined

In [ ]:
"""
create table m_hk_filter_v2_1 (
ifa string,
asn string,
latitude string,
longitude string,
bundle_id string,
carrier string,
timestamp string,
device_category string,
device_name string,
ip_address string,
major_os string,
city string,
connection_type string,
country string,
state string,
zip string,
platform string,
store_url string,
user_agent string
)
row format delimited fields terminated by ','
LOCATION 's3://alantuan/m_summary/m_hk_filter_v2/'
TBLPROPERTIES ("skip.header.line.count"="1")

"""